In [1]:
# ! pip install kafka-python

IP: 192.168.1.39

In [2]:
import json
import socket
import time
import numpy as np
from datetime import datetime

import pandas as pd

In [3]:
sensor_map: dict = {
    1: "gps_position",
    3: "accelerometer",
    4: "gyroscope",
    5: "magnetic_field",
    81: "orientation",
    82: "linear_acceleration",
    83: "gravity",
    84: "rotation_vector",
    85: "pressure",
    86: "battery_temperature",
}

In [4]:
def get_payload(row: list) -> dict:
    row: list = list(map(float, row))
    dict_: dict = {}
    for key in sensor_map:
        if key in row:
            _index: int = row.index(key)
            if key not in (85, 86):
                dict_[sensor_map[key]] = row[_index + 1 : _index + 4]
            else:
                dict_[sensor_map[key]] = row[_index + 1 : _index + 2][0]
    return dict_

def produce_magnitude(listt):
    magg = np.sqrt(listt[0]**2 + listt[1]**2 + listt[2]**2)
    return magg

In [5]:
import pandas as pd

df = pd.read_csv("athul_test.csv", index_col=0)
df.head(5)

,rotationRate,userAcceleration,act,id,weight,height,age,gender,trial
0,0.010253,0.006959,0.0,0.0,102.0,188.0,46.0,1.0,5.0
1,0.010920,0.010673,0.0,0.0,102.0,188.0,46.0,1.0,5.0
2,0.008377,0.007010,0.0,0.0,102.0,188.0,46.0,1.0,5.0
3,0.006555,0.014892,0.0,0.0,102.0,188.0,46.0,1.0,5.0
4,0.007724,0.013001,0.0,0.0,102.0,188.0,46.0,1.0,5.0


In [6]:
from sklearn.model_selection import train_test_split

Y = df['act']
X = df[['rotationRate', 'userAcceleration', 'weight', 'height', 'age', 'gender']]


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=52)

print('Shape of X_train:', X_train.shape)
print('Shape of X_test:', X_test.shape)
print('Shape of Y_train:', Y_train.shape)
print('Shape of Y_test:', Y_test.shape)

Shape of X_train: (664716, 6)
Shape of X_test: (166179, 6)
Shape of Y_train: (664716,)
Shape of Y_test: (166179,)


In [7]:
from sklearn.ensemble import RandomForestClassifier

model_mag_rd = RandomForestClassifier(n_estimators=100)
model_mag_rd.fit(X_train, Y_train)

RandomForestClassifier()

In [8]:
Y_pred = model_mag_rd.predict(X_test)
Y_pred

array([0., 5., 5., ..., 1., 4., 4.])

In [9]:
from sklearn.metrics import classification_report
target_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

print(classification_report(Y_test, Y_pred, target_names=target_names))

                    precision    recall  f1-score   support

           Walking       0.83      0.81      0.82     46770
  Walking Upstairs       0.80      0.83      0.81     44994
Walking Downstairs       0.26      0.17      0.20      9988
           Sitting       0.35      0.27      0.31     11700
          Standing       0.58      0.69      0.63     31753
            Laying       0.71      0.71      0.71     20974

          accuracy                           0.70    166179
         macro avg       0.59      0.58      0.58    166179
      weighted avg       0.69      0.70      0.69    166179



In [10]:
labels = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

In [11]:
UDP_IP: str = ""
UDP_PORT: int = 5555

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

In [12]:
def find_activity(weight, height, age, gender):

    data, addr = sock.recvfrom(1024)
    row: list = data.decode("utf-8").split(",")
        
    act = " "
    dt_string = " "

    if len(row) == 13:

        acc_mag = np.sqrt(float(row[2])**2 + float(row[3])**2 + float(row[4])**2)
        gyro_mag = np.sqrt(float(row[6])**2 + float(row[7])**2 + float(row[8])**2)
        magn_mag = np.sqrt(float(row[10])**2 + float(row[11])**2 + float(row[12])**2)

        data = [[gyro_mag, acc_mag, weight, height, age, gender]]
        
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

        out = model_mag_rd.predict(data)

        print("\nAcc Magnitude : ", acc_mag, "\nGyro Magnitude : ", gyro_mag, "\nMagneto Magnitude : ", magn_mag)
        
        act = labels[int(out)-1]
        print("Activity : ", act)

    return act, str(dt_string)
        

In [13]:
def choice(smoot, weight, height, age, gender, model):
    
    print("Operation is ", smoot)
    print(weight, height, age, gender, model)
    
    act = " Please wait"
    time_stamp = " "
    
    if smoot == "start" and model == "Model1":
        
        act, time_stamp = find_activity(weight, height, age, gender)
  
    return act, time_stamp

In [14]:
import gradio as gr


smoot = gr.inputs.Radio(["start"], label="Start")
weight = gr.inputs.Slider( minimum=30, maximum=120, step=1, default=70, label="Weight in Kg")
height = gr.inputs.Slider(minimum=100, maximum=200, step=1, default=178, label="Height in cm")

age = gr.inputs.Slider(minimum=20, maximum=100, step=1, default=24, label="Age")
gender = gr.inputs.Dropdown(choices = ["Female", "Male"], type="index", label="Gender")
model = gr.inputs.Dropdown(choices = ["Model1", "Model2"], type="value", label="Select Model")


act = gr.outputs.Textbox(type="auto", label="Activity")
time_stamp = gr.outputs.Textbox(type="auto", label="Time Stamp")

gr.Interface(choice, inputs = [smoot, weight, height, age, gender, model],   
             outputs = [act, time_stamp],
             live=True).launch(share=True)

IMPORTANT: You are using gradio version 1.4.0, however version 1.5.0 is available, please upgrade.
--------
Running locally at: http://127.0.0.1:7860/
This share link will expire in 24 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://27083.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://27083.gradio.app')

Operation is  start
30 100 20 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
67 183 23 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
30 100 20 0 Model1
Operation is  start
30 10

Operation is  start
76 176 20 0 Model1
Operation is  start
76 176 20 0 Model1
Operation is  start
76 176 20 0 Model1
Operation is  start
76 176 20 0 Model1
Operation is  start
76 176 20 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  10.004086864876774 
Gyro Magnitude :  0.0022360679774997894 
Magneto Magnitude :  38.821339878989235
Activity :  Walking
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
30 100 20 0 Model1

Acc Magnitude :  9.989391673170093 
Gyro Magnitude :  0.005 
Magneto Magnitude :  38.55035706708824
Activity :  Walking
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  10.013259659072066 
Gyro Magnitude :  0.005385164807134504 
Magneto Magnitude :  39.85059500183153
Activity :  Walking
Operation is  star


Acc Magnitude :  10.013740260262397 
Gyro Magnitude :  0.00282842712474619 
Magneto Magnitude :  39.89415318063538
Activity :  Walking
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  10.025627212299487 
Gyro Magnitude :  0.0037416573867739417 
Magneto Magnitude :  39.863585124772705
Activity :  Walking
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  10.02327416566064 
Gyro Magnitude :  0.003 
Magneto Magnitude :  39.18998174278728
Activity :  Walking
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start
76 176 51 0 Model1
Operation is  start

...


Acc Magnitude :  6.170748171818389 
Gyro Magnitude :  2.162462947659451 
Magneto Magnitude :  39.67889377238232
Activity :  Standing
Operation is  start
76 176 51 0 Model1
Operation is  start
30 100 20 0 Model1

Acc Magnitude :  7.410296687717706 
Gyro Magnitude :  0.854988888816691 
Magneto Magnitude :  39.73238672669942
Activity :  Standing
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  10.41347367596423 
Gyro Magnitude :  0.9447301201930633 
Magneto Magnitude :  39.78090108330881
Activity :  Standing

Acc Magnitude :  16.696780318372763 
Gyro Magnitude :  1.6724556795323458 
Magneto Magnitude :  39.92811192631077
Activity :  Standing
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  10.069814546455163 
Gyro Magnitude :  1.423147567893084 
Magneto Magnitude :  39.63227910428569
Activity :  Standing
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  4.682821158233571 
Gyro Magnitude :  4.546714088217996 
Magneto Magnitude :  39.06473366093771
Activity :  Standi

Acc Magnitude :  9.777747695660796 
Gyro Magnitude :  0.01746424919657298 
Magneto Magnitude :  38.69056771359138
Activity :  Standing
Operation is  start
76 176 51 0 Model1
Operation is  start
30 100 20 0 Model1

Acc Magnitude :  9.780777525329977 
Gyro Magnitude :  0.005999999999999999 
Magneto Magnitude :  39.20526756699921
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.78330951161211 
Gyro Magnitude :  0.020445048300260872 
Magneto Magnitude :  39.123174781707064
Activity :  Standing

Acc Magnitude :  9.790866611286255 
Gyro Magnitude :  0.005744562646538029 
Magneto Magnitude :  38.82453304290986
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.788341943352817 
Gyro Magnitude :  0.017578395831246946 
Magneto Magnitude :  38.92767681226302
Activity :  Standing
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.784884107642768 
Gyro Magnitude :  0.014456832294800959 
Magneto Magnitude :  39.60115819013378
Activity

Operation is  start
30 100 20 0 Model1
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.984578809343938 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.34867673759818
Activity :  Standing

Acc Magnitude :  9.976530308679466 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.03714427311506
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.976531361149526 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.32245905331964
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.984491424203839 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.7206815273857
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.978581913278058 
Gyro Magnitude :  0.002 
Magneto Magnitude :  39.383132912453775
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.984613112184167 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.262977676177336
Activity :  Walking
Operation is  start
76 176 51 0 Model1


Acc Magnitude :  9.993504740580255 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.216135352683594
Activity :  Standing

Acc Magnitude :  9.991573950084142 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.69499484821733
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.988565162224253 
Gyro Magnitude :  0.0022360679774997894 
Magneto Magnitude :  39.385313709554225
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.993446702714735 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.29719610862841
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.98752041299541 
Gyro Magnitude :  0.002 
Magneto Magnitude :  39.2660049788618
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.996576914124153 
Gyro Magnitude :  0.001 
Magneto Magnitude :  38.989179152682865
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.989456541774432 
Gyro Magnitude :  0.002 

Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.999447484736343 
Gyro Magnitude :  0.001 
Magneto Magnitude :  38.679691441375276
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.998506438463698 
Gyro Magnitude :  0.001 
Magneto Magnitude :  40.09656270804269
Activity :  Walking
Operation is  start
76 176 51 0 Model1
Operation is  start
30 100 20 0 Model1

Acc Magnitude :  9.993474771069371 
Gyro Magnitude :  0.001 
Magneto Magnitude :  38.851760462043416
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.994577229678102 
Gyro Magnitude :  0.0 
Magneto Magnitude :  39.515180829650774
Activity :  Standing

Acc Magnitude :  9.992508493866792 
Gyro Magnitude :  0.001 
Magneto Magnitude :  38.51170264997381
Activity :  Walking
Operation is  start
76 176 51 0 Model1

Acc Magnitude :  9.99257669472694 
Gyro Magnitude :  0.001 
Magneto Magnitude :  38.87972343008628
Activity :  Walking
Operation is  start
76 176 51 0 Model1
